In [ ]:
import os
import pandas as pd
import numpy as np
import pvlib
import math
import re
import subprocess

import plotly.express as px

from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

from sklearn.preprocessing import MinMaxScaler
from pykalman import KalmanFilter
from sklearn.impute import KNNImputer

# Kalman Smoothing using R objects
import rpy2.robjects as robjects
# import R packages
from rpy2.robjects.packages import importr

# Impute TS
imputeTS = importr('imputeTS') 
kalman_StructTs = robjects.r['na.kalman']
kalman_auto_arima = robjects.r['na.kalman']
sea_decom = robjects.r['na.seadec']
sea_split = robjects.r['na.seasplit']

# # Zoo
# zoo = importr('zoo') 
# kalman_StructTs_zoo = robjects.r['na.StructTS']
# nan_interpolation_sea = robjects.r['na.approx']

# # Forecast
# forecast = importr('forecast') 
# nan_interpolation_forecast = robjects.r['na.interp']

# import rpy2.robjects as ro
# from rpy2.robjects import pandas2ri
# pandas2ri.activate()


In [ ]:
def execute(file):
    path_list = []
    datapath = re.sub(r'Notebooks|Python Scripts','Data/',os.getcwd())
    for dir in os.scandir(datapath):
        if re.search(r'\.',dir.name): continue
        year_path = datapath + f"{dir.name}"
        for dir in os.scandir(year_path):
            if dir.name == file:
                month_path = year_path + f"/{dir.name}/"
                for dir in os.scandir(month_path):
                    if not re.search(r'\.csv|\.xlsx',dir.name): continue
                    path_list += [month_path + f"{dir.name}"]
    return path_list

In [ ]:
def path_func(year,month,year_2,month_2,path_list,file):
    path_found = []
    for path in path_list:
        path_copy = path.lower()
        data = re.search(r"/(\d{4})/[a-z]*/([a-z]*)\.",path_copy).group(1,2)
        if year:
            if not path_found:
                if re.search(fr"{year}",data[0]) and re.search(fr"{month}",data[1]):
                    path_found += [path]
            else:
                if re.search(fr"{year}",data[0]) and re.search(fr"{month}",data[1]):
                    path_found += [path]
                    path_found = [path_found[1],path_found[0]]

        if year_2:
            if re.search(fr"{year_2}",data[0]) and re.search(fr"{month_2}",data[1]):
                path_found += [path]

    return path_found

In [ ]:
def surrounding_month(month, year , cond):
    month_dict = {0:'dec',13:'jan',8:'aug',9:'sep',10:'oct',11:'nov',12:'dec',1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'jun',7:'jul'}
    if cond == 'prev':
        if month == 1:
            year -= 1
        return month_dict[month-1], year
    else:
        if month == 12:
            year += 1
        return month_dict[month+1], year

In [ ]:
def reshape_df(df,file):
    df['DayID'] = df['DayID'].astype(str)
    df['TimeID'] = df['TimeID'].astype(str)
    df['date'] = df['DayID'] + 'T' +  df['TimeID']
    df = df.drop(columns = ['DayID','TimeID'])
    df.date = pd.to_datetime(df.date)
    df = df.set_index('date')
    df.index = df.index.tz_localize(tz = 'Etc/UTC')
    df = df.sort_index()
    if file == 'Irradiance':
        df.columns = ['GlobalIR','DirectIR','DiffuseIR','WindSpeed','Temperature']
    else:
        df.columns = ['MonoSi_Vin','MonoSi_Iin','MonoSi_Vout','MonoSi_Iout','PolySi_Vin','PolySi_Iin','PolySi_Vout','PolySi_Iout','TFSi_a_Vin','TFSi_a_Iin','TFSi_a_Vout','TFSi_a_Iout','TFcigs_Vin','TFcigs_Iin','TFcigs_Vout','TFcigs_Iout','TempF_Mono','TempF_Poly','TempF_Amor','TempF_Cigs']
    return df

In [ ]:
def add_missing_times(df):
    
    # creating of list of times to find interval gaps
    time_list = list(df.index)
    
    # calculating interval gaps if > 21s and storing [interval length (s), start_time, end_time]
    missing_intervals = [[(time_list[time+1] - time_list[time]).total_seconds(),time_list[time],time_list[time+1]]
                 for time in range(len(time_list)-1) if (time_list[time+1] - time_list[time]).total_seconds() > 21]
    
    # generating timestamps to fill interval gaps 
    interval_list = [element for sublist in [pd.date_range(start=interval[1],
                             end=interval[2]-pd.Timedelta(1,'s'),
                             freq='11s') for interval in missing_intervals] for element in sublist]
    
    # checking for missing values at the beginning of the month
    if time_list[0] > time_list[0].replace(day=1,hour=1):
        print("Month found with missing values at the beginning of the month.")
        print('Time:',time_list[0])
        interval_list += [time for time in pd.date_range(start=time_list[0].replace(day=1,hour=0,minute=0,second=0),
                             end=time_list[0]-pd.Timedelta(1,'s'),
                             freq='11s')]
        missing_intervals += [[(time_list[0] - time_list[0].replace(day=1,hour=0,minute=0,second=0)).total_seconds(),
                             time_list[0].replace(day=1,hour=0,minute=0,second=0),time_list[0]]]
        
    # checking for missing values at the end of the month    
    next_month = time_list[0].replace(day=28,hour=0,minute=0,second=0) + pd.Timedelta(4,'d')
    last_day = next_month - pd.Timedelta(next_month.day,'d')
    if time_list[-1] < last_day.replace(hour = 23,minute=0):
        print("Month found with missing values at the end of the month.")
        print('Time:',time_list[-1])
        interval_list += [time for time in pd.date_range(start=time_list[-1],
                     end=last_day.replace(hour=23,minute=59,second=59),
                     freq='11s')]
        missing_intervals += [[(last_day.replace(hour=23,minute=59,second=59) - time_list[-1]).total_seconds(),
                             time_list[-1],last_day.replace(hour=23,minute=59,second=59)]]
        
    interval_list = list(set(interval_list))
    mt_df = pd.DataFrame(index=interval_list,columns=df.columns)
    mt_df.loc[interval_list] = np.nan
    df = pd.concat([df,mt_df], axis = 0).sort_index()

    return df

In [ ]:
def remove_night(df):
    lat = 49.102
    lon = 6.215
    alt = 220
    solpos = pvlib.solarposition.get_solarposition(
        time=df.index,latitude=lat,longitude=lon,altitude=alt,method='pyephem')
    df = df[solpos['zenith'] <= 90]
    return df

def irr(df):
    # Removing Temperature Values #
    df[df['Temperature'] > 60] = np.nan

    # Removing Wind Speed Values #
    df[df['WindSpeed'] > 100] = np.nan

    # Removing DirectIR Values #
    df[df['DirectIR'] > 2000] = np.nan

    # Removing DiffuseIR Values #
    df[df['DiffuseIR'] > 2000] = np.nan

    # Removing Negative Values #
    df[df < 0] = np.nan

    return df

def deg_fix(df):
    # Removing Negative Values #
    df[df < 0] = np.nan

    return df

In [ ]:
def df_cleaner(path_list,file):
    df_clean = pd.DataFrame()

    for path in path_list:

        df_load = pd.read_csv(path,sep="\t|,",engine='python')
        
        # === In case a file isn't stored properly or empty === #
        if df_load.empty:
            raise Exception("Loaded an empty dataframe")
        
        # ==== reshaping df for timestap & adjusted headers ==== #
        df_load = reshape_df(df_load)

        # === filling gaps in time intervals === #
        df_load = add_missing_times(df_load)

        # # ==== Using PvLib to remove nightime values === #
        df_load = remove_night(df_load)
        
        if file == 'Irradiance':
            # === Removing Values for Irradiance === #
            df = irr(df)

        else:
            # === Removing Values for Deger & Fixed === #
            df = deg_fix(df)

        df_clean = pd.concat([df_clean,df_load],axis=0,ignore_index=False).sort_index()
        
    return df_clean

In [ ]:
def prep(pre_df, df_2):
    
    pre_df = pre_df.dropna(axis=0)
    if 'GlobalIR' in pre_df.columns:
        pre_df,df_2 = pre_df.drop(['GlobalIR'],axis=1),df_2.drop(['GlobalIR'],axis=1)
    copy_df = pre_df.copy()
    pre_df, df_2 = pre_df.reset_index(), df_2.reset_index()
    pre_df, df_2 = pre_df.drop(['index'],axis=1),df_2.drop(['index'],axis=1)    
        
    # create a boolean mask that identifies NaN values
    nan_mask = df_2.isna() 
    # use np.where to find integer positions of NaN values
    nan_indices = list(np.where(nan_mask))
    scaler = MinMaxScaler(feature_range=(pre_df.index[0], pre_df.index[-1]))
    nan_indices[0] = scaler.fit_transform(nan_indices[0].reshape(-1, 1))
    nan_indices[0] = nan_indices[0].reshape(1,-1)[0].astype(int)
    
    nan_indices = tuple(zip(nan_indices))
        
    pre_df.iloc[nan_indices] = np.nan
    pre_df.index = copy_df.index    

    pre_df.to_csv('Data_Summary_Files/test_data.csv')
        
    return pre_df, copy_df

In [ ]:
def df_mask(pre_df, date, nan_gap):

    if not nan_gap:
        nan_gap = float('inf')
    else:
        if not re.match(r'\d*',nan_gap):
            raise Exception(f"Incorrect input NaN gap size: {nan_gap}")
        else:
            nan_gap = int(nan_gap)
    
    test_gaps = {}
    for col in range(len(pre_df.columns)):
        inx_ind = []
        test_gaps[pre_df.columns[col]] = []
        for index in range(len(pre_df.index)):
            if pre_df.index[index].month != int(date): continue
            if index in inx_ind: continue
            c = 0
            while np.isnan(pre_df.iloc[index+c,col]) and pre_df.iloc[index+c].name != pre_df.iloc[-1].name:
                inx_ind += [index+c]
                c += 1
            if not c: continue
            dt = (pre_df.index[index+c] - pre_df.index[index]).total_seconds()
            if dt <= nan_gap:
                test_gaps[pre_df.columns[col]] += inx_ind
        test_gaps[pre_df.columns[col]] = list(set(test_gaps[pre_df.columns[col]]))
        
    return nan_gap, test_gaps

In [ ]:
def checking_gaps(df, file):
    
    month_i = df.index[0].month

    if int(df.index[0].month) == 2 and int(df.index[0].year) == 2023:
        return False, False, month_i
        
    beg_ind = False
    end_ind = False
    path_list = execute(file)
    
    if df.iloc[0:int(len(df)/df.iloc[-1].name.day*5),:].isna().sum().sum() / df.iloc[0:int(len(df)/df.iloc[-1].name.day*5),:].size*100 > 5:
        if int(df.index[0].month) == 1 and int(df.index[0].year) == 2021:
            print("January 2021 is the first month observed can't join prior month.")
            return False, False, month_i
        month,year = surrounding_month(df.iloc[0].name.month,df.iloc[0].name.year ,'prev')
        path_list = path_func(year,month,'', '', path_list, file)
        load_beg = df_cleaner(path_list,file)
        if 'GlobalIR' in load_beg.columns:
            load_beg = load_beg.drop(['GlobalIR'],axis=1)
        beg_ind = True
        
    if df.iloc[int(len(df)/df.iloc[-1].name.day*(df.iloc[-1].name.day - 4)):-1, :].isna().sum().sum() / df.iloc[int(len(df)/df.iloc[-1].name.day*(df.iloc[-1].name.day - 4)):-1, :].size*100 > 5:
        if int(df.index[0].month) == 2 and int(df.index[0].year) == 2023:
            print("Feburary 2021 is the last month observed can't join following month.")
            return False, False, month_i
        month,year = surrounding_month(df.iloc[0].name.month, df.iloc[0].name.year, '')
        path_list = path_func(year, month, '', '', path_list, file)
        load_end = df_cleaner(path_list,file)
        if 'GlobalIR' in load_end.columns:
            load_end = load_end.drop(['GlobalIR'],axis=1)
        end_ind = True
    
    if beg_ind and end_ind:
        return load_beg, load_end, month_i
    elif beg_ind:
        return load_beg, False, month_i
    elif end_ind:
        return False, load_end, month_i
    else:
        return False, False, month_i

In [ ]:
def error_metric(imputed_df, copy_df, test_gaps):
    
    error_dict = {}
    
    for col in imputed_df.columns:
        
        pred_val = imputed_df[col].iloc[test_gaps[col]]
        test_val = copy_df[col].iloc[test_gaps[col]]

        mae = mean_absolute_error(test_val,pred_val)
        mse = mean_squared_error(test_val, pred_val)
        rmse = np.sqrt(mse)
        r2 = r2_score(test_val, pred_val)
        
        error_dict[col] = [mae,mse,rmse,r2]

        print(f"For {col}")
        print(f"Mean Absolute Error: {mae}")
        print(f"Mean Squared Error: {mse}")
        print(f"Root Mean Squared Error: {rmse}")
        print(f"R2 Score: {r2} \n")
        
    error_df = pd.DataFrame(error_dict).T
    error_df.columns = ['mae','mse','rmse','r2']
    
    print(error_df)
    
    px.bar(error_df, x = error_df.index, y = error_df['r2']).show()
    
    return error_df

In [ ]:
def main(year, month, year_2, month_2, file):
    if not re.search(r'\d{4}',year):
        raise Exception(f"Incorret Input: {year}")
    elif not re.search(r'[A-Za-z]{3}',month):
        raise Exception(f"Incorret Input: {month}")
    elif not re.search(r'[A-Za-z]{3}',month_2):
        raise Exception(f"Incorret Input: {month_2}")
    elif not re.search(r'\d{4}',year_2):
        raise Exception(f"Incorret Input: {year_2}")
    elif not [file_i for file_i in ['Irradiance','Deger','Fixed'] if re.search(fr'{file}',file_i)]:
        raise Exception(f"Incorret Input: File")
    else:
        path_list = execute(file)
        if not year and month and year_2 and month_2:
            df = df_cleaner(path_list,file)
            df_2 = pd.DataFrame()
        else:
            path_list = path_func(year,month,year_2,month_2,path_list,file)
            df = df_cleaner([path_list[0]],file)
            df_2 = df_cleaner([path_list[1]],file)
    return df, df_2,file, year_2, month_2, year, month

# Load all the Data
df_load, df2_load, file, year_2, month_2, year, month = main(year = input("Year (format: YYYY): "),month = input("Month (format: jul): "),
     year_2 = input("Second Year (format: YYYY): "),month_2 = input("Second Month (format: jul): "),
     file = input("File (opt: Irradiance/Deger/Fixed): "))

In [ ]:
def pre_processesing(df_final, df2_load, file):
    
    # Clean all the data
    df_final, copy_final = prep(df_load.copy(), df2_load.copy())

    beg_df, end_df, month_i = checking_gaps(df_final.copy(), file)

    if type(beg_df) == type(df_final) and type(end_df) == type(df_final):
        df = pd.concat([beg_df,df_final,end_df],axis=0,ignore_index=False)
        copy_final =  pd.concat([beg_df,copy_final,end_df],axis=0,ignore_index=False)

    #     index = np.hstack([list(beg_df.index.values),list(copy_final.index.values),list(end_df.index.values)])
    #     df.index = index

    elif type(beg_df) == type(df_final):
        df = pd.concat([beg_df,df_final],axis=0,ignore_index=False)
        copy_final =  pd.concat([beg_df,copy_final],axis=0,ignore_index=False)

    #     index = np.hstack([list(beg_df.index.values),list(copy_final.index.values)])
    #     df.index = index

    elif type(end_df) == type(df_final):
        df = pd.concat([df_final,end_df],axis=0,ignore_index=False)
        copy_final =  pd.concat([copy_final,end_df],axis=0,ignore_index=False)

    #     index = np.hstack([np.array(end_df.index.values),np.array(copy_final.index.values)])
    #     df.index = index


    nan_gap, test_gaps = df_mask(df.copy(), month_i, nan_gap = input('Include gaps smaller than (seconds): '))

    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR', title = 'Test Dataframe').show()
        px.scatter(copy_final,x=copy_final.index,y='DirectIR', title = 'Observed Dataframe').show()

    total_nan = df.isna().sum().sum()
    total_values = df.size
    mt_count = df.isna().all(axis=1).sum()
    t_perc = round(total_nan/total_values * 100,3)
    mt_perc = round(mt_count*5/total_values * 100,3)

    print("Test Dataframe NaN Summary: \n")

    print(f"Percentage of NaN values due to System Outage: {mt_perc}% \n")

    print(f"Precentage of MAR NaN values: {round(t_perc-mt_perc,3)}% \n")

    print(f"Precentage of Total NaN values: {t_perc}% \n")

    total_nan = copy_final.isna().sum().sum()
    total_values = copy_final.size
    mt_count = copy_final.isna().all(axis=1).sum()
    t_perc = round(total_nan/total_values * 100,3)
    mt_perc = round(mt_count*5/total_values * 100,3)

    print("Observed Dataframe NaN Summary: \n")

    print(f"Percentage of NaN values due to System Outage: {mt_perc}% \n")

    print(f"Precentage of MAR NaN values: {round(t_perc-mt_perc,3)}% \n")

    print(f"Precentage of Total NaN values: {t_perc}%")
    
    return df_final, copy_final, nan_gap, test_gaps
    
    
df_final, copy_final, nan_gap, test_gaps = pre_processesing(df_load,df2_load,file)


In [ ]:
def fill_forward(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df.iloc[[-1,0],:] = 0

    for col in df.columns:
        
        if not df[col].isna().sum().sum(): continue

        df[col] = df[col].fillna(method='ffill')
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df
    
imputation_dict['LOCF'] = fill_forward(df.copy()).to_dict()

In [ ]:
def nearest(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df.iloc[[-1,0],:] = 0
    
    for col in df.columns:
        
        if not df[col].isna().sum().sum(): continue

        df[col] = df[col].interpolate(method='nearest')
            
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

# imputation_dict['Nearest Neighbor'] = nearest(df.copy()).to_dict()

In [ ]:
def interpolate_linear(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    for col in df.columns:
        
        if not df[col].isna().sum().sum(): continue

        df[col] = df[col].interpolate(method='linear', limit_direction = 'both')
            
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
    
    return error_df

# imputation_dict['Linear Interpolation'] = interpolate_linear(df.copy()).to_dict()



In [ ]:
def kalman(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    for col in df.columns:
        
        arr = np.ndarray.tolist(df[col].values)
        arr = robjects.FloatVector(arr)

        df[col] = kalman_StructTs(arr, model = "StructTS")
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
    
    return error_df

# imputation_dict['Kalman Smoothing'] = kalman(df.copy()).to_dict()

In [ ]:
def ARIMA(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    for col in df.columns:
        
        arr = np.ndarray.tolist(df[col].values)
        arr = robjects.FloatVector(arr)

        df[col] = kalman_auto_arima(arr, model = "auto.arima")
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

# imputation_dict['ARIMA'] = ARIMA(df.copy()).to_dict()

In [ ]:
def seasonal_decom(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    for col in df.columns:
        
        arr = np.ndarray.tolist(df[col].values)
        arr = robjects.FloatVector(arr)

        df[col] = sea_decom(arr, algorithm = "kalman")
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

# imputation_dict['Seasonal Decomposition'] = seasonal_decom(df.copy()).to_dict()


In [ ]:
def knn(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df['Seconds'] = [(time - time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() for time in df.index]
    df['Day'] = [d.day for d in df.index]
    
    for col in df.columns:
        if df[col].isna().sum():
            df_KNN = df[[col,'Day', 'Seconds']].copy()
            scaler = MinMaxScaler()
            scaled_df = pd.DataFrame(scaler.fit_transform(df_KNN), columns = df_KNN.columns)
            imputer = KNNImputer(n_neighbors=7,weights='distance')
            knn_solar = pd.DataFrame(imputer.fit_transform(scaled_df),
                                    columns=scaled_df.columns)
            inverse_knn_solar = pd.DataFrame(scaler.inverse_transform(knn_solar),
                                columns=knn_solar.columns, index=df_KNN.index)
            df[col] = inverse_knn_solar[col]
            
    error_df = error_metric(df.drop(['Seconds','Day'], axis = 1), copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

# imputation_dict['K-Nearest Neighbor'] = knn(df.copy()).to_dict()


In [ ]:
def zoo_functions(df):
    
    df.iloc[[-1,0],:] = 0
    
    df['seconds'] = [(df.index[-1] - time).total_seconds() for time in list(df.index)]
    
    df.index.name = 'Timestamp'
    
    df.to_csv('R_df_.csv')
    
    cwd = os.getcwd()
    
    subprocess.call("Rscript " + cwd + "/r_imputation.R", shell=True)

def zoo_spline(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df = pd.read_csv('spline_df.csv')
    
    df['Timestamp'] = copy_final.index
    
    df = df.set_index('Timestamp')
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

def forecast_interpolate(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df = pd.read_csv('forecast_int_df.csv')
    
    df['Timestamp'] = copy_final.index
    
    df = df.set_index('Timestamp')
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

def zoo_interpolation(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df = pd.read_csv('interpolation_df.csv')
    
    df['Timestamp'] = copy_final.index
    
    df = df.set_index('Timestamp')
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

def forecast_auto_arima(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df = pd.read_csv('forecast_arima.csv')
    
    df['Timestamp'] = copy_final.index
    
    df = df.set_index('Timestamp')
    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
        
    return error_df

zoo_functions(df.copy())

# imputation_dict['Forecast Auto.Arima'] = forecast_auto_arima(df_final.copy()).to_dict()
imputation_dict['Zoo Interpolation'] = zoo_interpolation(df.copy()).to_dict()
imputation_dict['Zoo Spline'] = zoo_spline(df.copy()).to_dict()
imputation_dict['Forecast Interpolation'] = forecast_interpolate(df.copy()).to_dict()

In [ ]:
def get_int_loc(index,column,df):
    i = df.index.get_loc(index)
    
    if column == 'DiffuseIR':
        c = 0
    elif column == 'DirectIR':
        c = 1
    elif column == 'WindSpeed':
        c = 2
    elif column == 'Temperature':
        c = 3

    return [i,c]    

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

def get_next_good_index(index,column,df):
    while np.isnan(df.loc[index,column]):
        index.replace(day=(index.day + 1))
        index = nearest(df.index, index)
        if index.day > df.index.day[-1]:
            raise Exception('Out of Bounds Error')
    
    return index

def get_previous_good_index(index,column,df):
    while np.isnan(df.loc[index,column]):
        index.replace(day=(index.day - 1))
        index = nearest(df.index, index)
        if index.day < 1:
            raise Exception('Out of Bounds Error')
        
    return index
    
def small_hole_interpolation(index,column,df):
    [i,c] = get_int_loc(index,column,df)
    lasti = i-1
    while np.isnan(df.iloc[lasti,c]):
        lasti -= 1
    nexti = i+1
    while np.isnan(df.iloc[nexti,c]):
        nexti += 1
    last = df.iloc[lasti,c]
    next = df.iloc[nexti,c]
    dt1 = (df.index[nexti] - df.index[lasti])/np.timedelta64(1,'s')
    m = (next-last)/dt1
    dt2 = (df.index[i] - df.index[lasti])/np.timedelta64(1,'s')
    new_value = last + m*dt2
    return new_value


def last_hole(index,column,df):
    [i,c] = get_int_loc(index,column,df)
    
    l = 1
    
    while np.isnan(df.iloc[i+1,c]):
        l = l + 1
        
        if i < len(df):
            i = i + 1
        else:
            break
    
    return [df.index[i],l]

def interpolate(c,df):
    for j in df.index:
        if np.isnan(df.loc[j][c]):
            [i,l] = last_hole(j,c,df)
            dt = (i - j)/np.timedelta64(1,'s')
            [start,_] = get_int_loc(j,c,df)
            for x in range(start,start+l):
                new_value = small_hole_interpolation(df.index[x],c,df)
                df.at[df.index[x],c] = new_value
            
    return df[c]

def other_fun(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    if df.iloc[[0,-1],:].isna().values.any():
        
        df.iloc[[0,-1],:] = 0
    
    for col in df.columns:
        
        if not df[col].isna().sum(): continue
            
        df[col] = interpolate(col,df)
                    
    error_df = error_metric(df, copy_final, test_gaps)
    
    if 'DiffuseIR' in df.columns:
        px.scatter(df,x=df.index,y='DirectIR').show()
    
    return error_df

# imputation_dict['Prior Function'] = other_fun(df.copy()).to_dict()

In [ ]:
# Import necessary libraries
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Define function to impute missing values using MLP
def impute_missing_values(df, prediction_window=24, num_hidden_layers=1, num_neurons=10):
    
    df['Seconds'] = [(time - time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() for time in df.index]
    df['Day'] = [d.day for d in df.index]
    df['Month'] = [d.month for d in df.index]
    
    # Create a copy of the data to avoid modifying the original
    imputed_data = df.copy()

    # Create a list of the columns that have missing values
    missing_cols = imputed_data.columns[imputed_data.isnull().any()]

    # Loop through each column with missing values
    for col in missing_cols:
        # Create a copy of the data for the current column
        col_data = imputed_data[[col]].copy()

        # Create a new column for each of the features of time
        for i in range(prediction_window):
            col_data['t-'+str(prediction_window-i)] = col_data[col].shift(prediction_window-i)

        # Drop any rows with missing values
        col_data.dropna(inplace=True)

        # Split the data into training and testing sets
        X_train = col_data.drop(columns=[col])
        y_train = col_data[[col]]
        X_test = imputed_data.loc[imputed_data[col].isnull(), :].drop(columns=[col])

        # Standardize the data
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Create the MLP and fit it to the training data
        mlp = MLPRegressor(hidden_layer_sizes=(num_neurons,) * num_hidden_layers, activation='tanh')
        mlp.fit(X_train_scaled, y_train)

        # Predict the missing values and add them to the imputed data
        imputed_data.loc[imputed_data[col].isnull(), col] = mlp.predict(X_test_scaled)

    return imputed_data

# Call the impute_missing_values function and save the imputed data to a new CSV file
imputation_dict['Prior Function'] = impute_missing_values(df.copy()).to_dict()

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

def MLP(df, copy_final = copy_final.copy(), test_gaps=test_gaps):
    
    df = df.dropna()
    
    col_names = [col for col in df.columns]
    
    df['Seconds'] = [(time - time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() for time in df.index]
    df['Day'] = [d.day for d in df.index]
    df['Month'] = [d.month for d in df.index]
    
    for col in df.columns:
        
        if col in ['Seconds','Day','Month']: continue
        
        print(f'\nFor target variable: {col}')

        # Load the dataset and split into training and testing sets
        X = df.drop(col_names, axis = 1).to_numpy() # Features matrix with shape (n_samples, n_features)
        y = df[col].to_numpy() # Target vector with shape (n_samples,)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize the training and testing data
        scaler = StandardScaler()
        X_train_std = scaler.fit_transform(X_train)
        X_test_std = scaler.transform(X_test)

        # Create a Lasso model for feature selection
        lasso = Lasso(alpha=0.1)
        lasso.fit(X_train_std, y_train)

        # Print the coefficients of the Lasso model
        print("Coefficients:", lasso.coef_)

        # Print the indices of the selected features
        selected_indices = np.where(lasso.coef_ >= 0)[0]
        if not selected_indices.any(): 
            (print(f'No features meet current criteria for: {col}'))
            continue
        print("Selected indices:", df.drop(col_names, axis = 1).columns[selected_indices])

        # Train an MLP regressor with the selected features
        X_train_selected = X_train[:, selected_indices]
        X_test_selected = X_test[:, selected_indices]
        mlp = MLPRegressor(hidden_layer_sizes=(100,50), activation='tanh', solver='adam')
        mlp.fit(X_train_selected, y_train)

        # Evaluate the performance of the MLP regressor
        score = mlp.score(X_test_selected, y_test)
        print("Test score:", score)

MLP(df)
# imputation_dict['MLP'] = MLP(df.copy()).to_dict()

In [ ]:
from sklearn.model_selection import GridSearchCV

def MLP_grid(df):
    
    df = df.dropna()
    
#     col_names = [col for col in df.columns]
    
    df['Seconds'] = [(time - time.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds() for time in df.index]
    df['Day'] = [d.day for d in df.index]
#     df['Month'] = [d.month for d in df.index]
    
    for col in df.columns:
        
        if col in ['Seconds','Day','Month']: continue
        
        print(f'\nFor target variable: {col}')

        # Load the dataset and split into training and testing sets
        X = df.drop([col], axis = 1).to_numpy() # Features matrix with shape (n_samples, n_features)
        y = df[col].to_numpy() # Target vector with shape (n_samples,)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize the training and testing data
        scaler = StandardScaler()
        X_train_std = scaler.fit_transform(X_train)
        X_test_std = scaler.transform(X_test)

        # Create a Lasso model for feature selection
        lasso = Lasso(alpha=0.1)
        lasso.fit(X_train_std, y_train)

        # Print the coefficients of the Lasso model
        print("Coefficients:", lasso.coef_)

        # Print the indices of the selected features
        selected_indices = np.where(lasso.coef_ > 0)[0]
        if not selected_indices.any(): 
            (print(f'No features meet current criteria for: {col}'))
            continue
        print("Selected indices:", df.drop([col], axis = 1).columns[selected_indices])

        # Train an MLP regressor with the selected features
        X_train_selected = X_train[:, selected_indices]
        X_test_selected = X_test[:, selected_indices]

        # Define the hyperparameter grid to search over
        param_grid = {
            'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50)],
            'activation': ['tanh', 'relu'],
            'solver': ['adam', 'sgd'],
        }

        # Create an MLPRegressor with default hyperparameters
        mlp = MLPRegressor()

        # Use grid search to find the best hyperparameters
        grid_search = GridSearchCV(mlp, param_grid, cv=5)
        grid_search.fit(X_train_selected, y_train)

        # Print the best hyperparameters and the test score of the best model
        print("Best hyperparameters:", grid_search.best_params_)
        print("Test score:", grid_search.score(X_test_selected, y_test))
        
MLP_grid(df.copy())

In [ ]:
def performance(impute_dict):
    imputation_df = pd.DataFrame.from_dict({(outerKey, innerKey): values for outerKey, innerDict in impute_dict.items() for innerKey, values in innerDict.items()}).T
    r2_df = pd.DataFrame(imputation_df.drop(['mae','mse','rmse'], axis=0,level=1).max(axis=0),columns = ['r2'])
    r2_df['Method'] = imputation_df.drop(['mae','mse','rmse'], axis=0,level=1).idxmax(axis=0).values
    mae_df = pd.DataFrame(imputation_df.drop(['r2','mse','rmse'], axis=0,level=1).min(axis=0),columns = ['mae'])
    mae_df['Method'] = imputation_df.drop(['r2','mse','rmse'], axis=0,level=1).idxmin(axis=0).values
    rmse_df = pd.DataFrame(imputation_df.drop(['r2','mse','mae'], axis=0,level=1).min(axis=0),columns = ['rmse'])
    rmse_df['Method'] = imputation_df.drop(['r2','mse','mae'], axis=0,level=1).idxmin(axis=0).values
    for row in r2_df.index:
        print(row)
        print(f"Optimal Imputation Method for {row}: {r2_df.loc[row]['Method'][0]}, R2 score: {round(r2_df.loc[row]['r2'],5)}")
        print(f"Optimal Imputation Method for {row}: {mae_df.loc[row]['Method'][0]}, MAE score: {round(mae_df.loc[row]['mae'],5)}")
        print(f"Optimal Imputation Method for {row}: {rmse_df.loc[row]['Method'][0]}, RMSE score: {round(rmse_df.loc[row]['rmse'],5)}\n")

    return imputation_df

print(f"\nImputation methods for {file} and error metrics in {year}, {month} modeled with NaN values in {year_2}, {month_2}.\nIncluding gaps smaller than {nan_gap} seconds.\n")
performance(imputation_dict.copy())